In [8]:
import pandas as pd

# add the mcu name to dataframe
mcus = ["esp8266", "esp32", "esp32s2", "esp32c3", "nano33iot", "rpi_pico" ]

df = pd.read_csv('data/esp8266.csv')
df.insert(1, "mcu", "esp8266", True)

d = pd.read_csv('data/esp32.csv')
d.insert(1, "mcu", "esp32", True)
df = df.append(d, ignore_index = True)

d = pd.read_csv('data/esp32s2.csv')
d.insert(1, "mcu", "esp32s2", True)
df = df.append(d, ignore_index = True)

d = pd.read_csv('data/esp32c3.csv')
d.insert(1, "mcu", "esp32c3", True)
df = df.append(d, ignore_index = True)

d = pd.read_csv('data/nano33iot.csv')
d.insert(1, "mcu", "nano33iot", True)
df = df.append(d, ignore_index = True)

d = pd.read_csv('data/rpi_pico.csv')
d.insert(1, "mcu", "rpi_pico", True)
df = df.append(d, ignore_index = True)

dataObjectNames = df.name.unique()
print("list of Data Object")
print(dataObjectNames)
print("Number of row: ", len(df.index))

df.head()

# The next few charts will charts are "explorative", they are intented to give an initial gimpse about the collected data

list of Data Object
['obj int' 'array string' 'array int' 'obj null' 'obj random' 'Msgpack'
 'HA' 'json.org' 'WoT' 'Tasmota']
Number of row:  60


,name,mcu,deserTimeJson,deserTimeMsgPack,serTimeJson,serTimeMsgPack,serSpaceJson,serSpaceMsgPack
0,obj int,esp8266,226,184,298,115,48,35
1,array string,esp8266,137,109,83,46,74,58
2,array int,esp8266,100,62,112,34,45,25
3,obj null,esp8266,27,19,16,5,12,6
4,obj random,esp8266,6520,5297,2182,648,1970,1718


In [9]:
# Starts with more advanced charts (graph_objects instead express)
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS
from plotly.subplots import make_subplots
import os

In [10]:
if not os.path.exists("images"):
    os.mkdir("images")

In [11]:
fig = go.Figure()
i=0
for mcu in mcus:
    currentData = df[df.mcu==mcu]
    fig.add_trace(
        go.Bar(
            x=currentData.name,
            y=currentData.deserTimeMsgPack,
            name='MessagePack',
            legendgroup="myGroup",
            showlegend=False
        )
    )
    i = i+1

fig.update_layout(
    height=500,
    width=1100,
    title = "Deserialization time using MessagePack"
)
fig.update_yaxes(
    title="Time [us]",
    tickformat="d"
)
fig.show()

In [12]:
# charts groups primarly by MCU, compare MessagePack vs JSON deserialization time

cols=6
fig = make_subplots(
    cols=cols,
    subplot_titles=mcus,
    y_title='Time [us]',
)

for mcu in mcus:
    currentData = df[df.mcu==mcu]
    fig.add_trace(
        go.Bar(
            x=currentData.name,
            y=currentData.deserTimeJson,
            legendgroup="myGroup",
            showlegend=True if i==0 else False
        ),
        col = i%cols+1,
        row = 1
    )
    i = i+1

fig.update_layout(
    height=400, 
    width=1200,
    title = "Deserialization time using MessagePack and JSON for each MCUs",
)

fig.update_yaxes(
    tickformat="d",
)
fig.show()

# OBSERVATIONS
# The time scale is not uniform, but this is intended to show the relative similarities among MCUS 

In [13]:
# charts groups primarly by MCU, compare MessagePack vs JSON deserialization time
# It is the same of the previous chart 
rows = 2
cols = 3

fig = make_subplots(
    rows=rows,
    cols=cols,
    subplot_titles=mcus,
    vertical_spacing=0.3,
    y_title='Time [us]',
)
i=0
for mcu in mcus:
    currentData = df[df.mcu==mcu]
    fig.add_trace(
        go.Bar(
            x=currentData.name,
            y=currentData.deserTimeMsgPack,
            name='MessagePack',
            marker_color=DEFAULT_PLOTLY_COLORS[0],
            legendgroup="myGroup",
            showlegend=True if i==0 else False
        ),
        row = i//cols+1,
        col = i%cols+1
    )
    
    fig.add_trace(
        go.Bar(
            x=currentData.name,
            y=currentData.deserTimeJson,
            name='Json',
            marker_color=DEFAULT_PLOTLY_COLORS[1],
            legendgroup="myGroup",
            showlegend=True if i==0 else False
        ),
        row = i//cols+1,
        col = i%cols+1
    )
    i = i+1

fig.update_layout(
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1, # gap between bars of the same location coordinate.
    height=800,
    title="Deserialization time using MessagePack and JSON for each MCUs"
)
fig.update_yaxes(
    tickformat="d"
)
fig.show()

# OBSERVATIONS
# Each subchart is very similar (apart the different y scale)

In [14]:
# charts groups primarly by Data object, compare MessagePack vs JSON
from plotly.subplots import make_subplots

rows = 2
cols = 5

fig = make_subplots(
    rows= rows,
    cols= cols,
    subplot_titles=dataObjectNames,
    vertical_spacing=0.3,
    y_title='Time [us]',
)

i=0
for name in dataObjectNames:
    currentData = df[df.name==name]
    fig.add_trace(
        go.Bar(
            x=currentData.mcu,
            y=currentData.deserTimeMsgPack,
            name='MessagePack',
            marker_color=DEFAULT_PLOTLY_COLORS[0],
            legendgroup="myGroup",
            showlegend=True if i==0 else False
        ),
        row = i//cols+1,
        col = i%cols+1
    )
    
    fig.add_trace(
        go.Bar(
            x=currentData.mcu,
            y=currentData.deserTimeJson,
            name='Json',
            marker_color=DEFAULT_PLOTLY_COLORS[1],
            legendgroup="myGroup",
            showlegend=True if i==0 else False
        ),
        row = i//cols+1,
        col = i%cols+1
    )
    i = i+1

fig.update_layout(
    barmode='group',
    # gap between bars of adjacent location coordinates
    bargap=0.15,
    # gap between bars of the same location coordinate
    bargroupgap=0.1,
    height=700,
    width=1200,
    title="Deserialization time using MessagePack and JSON for each data object"
)
fig.update_yaxes(
    tickformat="d"
)
fig.show()

In [15]:
# charts groups primarly by MCU, compare MessagePack vs JSON serialization time
from plotly.subplots import make_subplots
rows = 2
cols = 3
fig = make_subplots(
    rows=rows,
    cols=cols,
    subplot_titles=mcus,
    vertical_spacing=0.3,
    y_title='Time [us]',
)
i=0


for mcu in mcus:
    currentData = df[df.mcu==mcu]
    fig.add_trace(
        go.Bar(
            x=currentData.name,
            y=currentData.serTimeMsgPack,
            name='MessagePack',
            marker_color=DEFAULT_PLOTLY_COLORS[0],
            legendgroup="myGroup",
            showlegend=True if i==0 else False
        ),
        row = i//cols+1,
        col = i%cols+1
    )
    
    fig.add_trace(
        go.Bar(
            x=currentData.name,
            y=currentData.serTimeJson,
            name='Json',
            marker_color=DEFAULT_PLOTLY_COLORS[1],
            legendgroup="myGroup",
            showlegend=True if i==0 else False
        ),
        row = i//cols+1,
        col = i%cols+1
    )
    i = i+1

fig.update_layout(
    title="Serialization time using MessagePack and JSON for each MCU",
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1, # gap between bars of the same location coordinate.
    height=800
)
fig.show()

# OBSERVATIONS
# Each subchart is very similar (apart the different y scale)

In [16]:
# charts groups primarly by Data object, compare MessagePack vs JSON serialization time
from plotly.subplots import make_subplots

rows = 2
cols = 5

fig = make_subplots(
    rows = rows,
    cols = cols,
    subplot_titles=dataObjectNames,
    vertical_spacing=0.3,
    y_title='Time [us]',
)

i=0
for name in dataObjectNames:
    currentData = df[df.name==name]
    fig.add_trace(
        go.Bar(
            x=currentData.mcu,
            y=currentData.serTimeMsgPack,
            name='MessagePack',
            marker_color=DEFAULT_PLOTLY_COLORS[0],
            legendgroup="myGroup",
            showlegend=True if i==0 else False
        ),
        row = i//cols+1,
        col = i%cols+1
    )
    
    fig.add_trace(
        go.Bar(
            x=currentData.mcu,
            y=currentData.serTimeJson,
            name='Json',
            marker_color=DEFAULT_PLOTLY_COLORS[1],
            legendgroup="myGroup",
            showlegend=True if i==0 else False
        ),
        row = i//cols+1,
        col = i%cols+1
    )
    i = i+1

fig.update_layout(
    title="Serialization time using MessagePack and JSON for each data object",
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1, # gap between bars of the same location coordinate.
    height=700,
    width=1100
)
fig.show()

fig.write_image("images/serialization_complete.svg")

In [17]:
import statistics

def calculateRelative(vf,vi):
    return (vf-vi)/vi

df['relativeSpace'] = df.apply(lambda x: -calculateRelative(x['serSpaceMsgPack'], x['serSpaceJson']), axis=1)

fig = go.Figure()
currentData = df[df.mcu==mcus[0]]
fig.add_trace(
    go.Bar(
        x=currentData.name,
        y=currentData.relativeSpace,
        name='MessagePack',
        marker_color=DEFAULT_PLOTLY_COLORS[0],
    )
)


fig.update_layout(
    title="Relative space saving of serialized object in MessagePack over JSON ",
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1, # gap between bars of the same location coordinate.
)
fig.update_yaxes(
    title='Space',
    tickformat="%"
)
fig.show()

fig.write_image("images/space.svg")

print("Average improvement: " + str(round(statistics.mean(currentData.relativeSpace)*100,2)) + "%") 

# OBSERVATION
# Note that "real-life" data object are the latter 3 (mixed and long content), where the mean improvement is about 15%.
# In very specific case to can get higher results

Average improvement: 25.17%


In [27]:
df['relativeTimeDeser'] = df.apply(lambda x: -calculateRelative(x['deserTimeMsgPack'], x['deserTimeJson']), axis=1)
df['relativeTimeSer'] = df.apply(lambda x: -calculateRelative(x['serTimeMsgPack'], x['serTimeJson']), axis=1)

fig = go.Figure()
currentData = df.groupby(['name'])[["relativeTimeDeser", "relativeTimeSer"]].mean().reindex(dataObjectNames)
print(currentData)
fig.add_trace(
    go.Bar(
        x=currentData.index,
        y=currentData.relativeTimeDeser,
        name='Deserialization',
        marker_color=DEFAULT_PLOTLY_COLORS[0],
    )
)
fig.add_trace(
    go.Bar(
        x=currentData.index,
        y=currentData.relativeTimeSer,
        name='Serialization',
        marker_color=DEFAULT_PLOTLY_COLORS[1],
    )
)


fig.update_layout(
    title="Relative improvement of (de)serialization time of MessagePack over JSON",
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1, # gap between bars of the same location coordinate.
)
fig.update_yaxes(
    title="Time",
    tickformat="%"
)
fig.show()


fig.write_image("images/time.svg")

print("Average improvement of deserialization time: " + str(round(statistics.mean(currentData.relativeTimeDeser)*100,2)) + "%") 
print("Average improvement of serialization time: " + str(round(statistics.mean(currentData.relativeTimeSer)*100,2)) + "%") 


              relativeTimeDeser  relativeTimeSer
name                                            
obj int                0.395709         0.524735
array string           0.267550         0.351165
array int              0.417810         0.315662
obj null               0.309126         0.371673
obj random             0.138956         0.708627
Msgpack                0.469163         0.427076
HA                     0.255665         0.653985
json.org               0.214252         0.555252
WoT                    0.229719         0.563630
Tasmota                0.152756         0.647583


Average improvement of deserialization time: 28.51%
Average improvement of serialization time: 51.19%
